In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import pandas as pd
import numpy as np
import pickle

In [ ]:
# Load csv file
df = pd.read_csv('./customer_churn_modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# Train and test data split
X = df.drop(['Exited', 'CustomerId', 'RowNumber', 'Surname'], axis=1)
y = df['Exited']

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = ['Geography', 'Gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Scaling and Encoding
sc = StandardScaler()
oe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_num = sc.fit_transform(X_train[numeric_features])
X_test_num = sc.transform(X_test[numeric_features])

X_train_cat = oe.fit_transform(X_train[categorical_features])
X_test_cat = oe.transform(X_test[categorical_features])

In [ ]:
# Save as pickle files
with open('scaler.pkl', 'wb') as f:
    pickle.dump(sc, f)

with open('encoder.pkl', 'wb') as f:
    pickle.dump(oe, f)

In [ ]:
# Concatenating numerical and categorical features
X_train = np.concatenate([X_train_num, X_train_cat], axis=1)
X_test = np.concatenate([X_test_num, X_test_cat], axis=1)

In [ ]:
# Build ANN
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Set up callbacks
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    validation_data=(X_test, y_test),
    callbacks=[callback, early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8374 - loss: 0.3846 - val_accuracy: 0.8555 - val_loss: 0.3464
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8551 - loss: 0.3519 - val_accuracy: 0.8400 - val_loss: 0.3668
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8596 - loss: 0.3464 - val_accuracy: 0.8650 - val_loss: 0.3367
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8593 - loss: 0.3422 - val_accuracy: 0.8600 - val_loss: 0.3416
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8595 - loss: 0.3417 - val_accuracy: 0.8615 - val_loss: 0.3428
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8636 - loss: 0.3354 - val_accuracy: 0.8695 - val_loss: 0.3383
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8635 - loss: 0.3358 - val_accuracy: 0.8640 - val_loss: 0.3393
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8610 - loss: 0.3346 - val_accu

In [ ]:
# Save the model
model.save('model.h5')